# Beautiful Soup Lyric Retrieval
## GOAL:
#### Retrieve lyrics from the links populated in the 'lyricurl' column of song_data.csv

In [1]:
# Import Libraries
import pandas as pd
from bs4 import BeautifulSoup as soup
from bs4 import NavigableString
from urllib.request import Request, urlopen
import os
import re
from tqdm import tqdm

In [2]:
data = pd.read_csv('data/song_data.csv', index_col=0)

In [7]:
data.head(10)

,Song,SongID,Popularity,Lyrics,href,lyricurl
Artist,,,,,,
Miley Cyrus,Flowers,0yLdNVWF3Srea0uzk55zFn,87.0,"We were good, we were goldKinda dream that can...",https://api.spotify.com/v1/tracks/0yLdNVWF3Sre...,https://genius.com/Miley-cyrus-flowers-lyrics
Metro Boomin,Creepin' (with The Weeknd & 21 Savage),2dHHgzDwk4BJdRwy9uXhTO,97.0,"Ooh, ooh-oohOoh-ooh-ooh, ooh, ooh-ooh (Just ca...",https://api.spotify.com/v1/tracks/2dHHgzDwk4BJ...,https://genius.com/Metro-boomin-the-weeknd-and...
SZA,Kill Bill,1Qrg8KqiBpW07V7PNxwwwL,93.0,I'm still a fan even though I was saltyHate to...,https://api.spotify.com/v1/tracks/1Qrg8KqiBpW0...,https://genius.com/Sza-kill-bill-lyrics
Central Cee,LET GO,3zkyus0njMCL6phZmNNEeN,94.0,"Well, you only need the light when it's burnin...",https://api.spotify.com/v1/tracks/3zkyus0njMCL...,https://genius.com/Central-cee-let-go-lyrics
Tiësto,10:35,6BePGk3eCan4FqaW2X8Qy3,90.0,NaN,https://api.spotify.com/v1/tracks/6BePGk3eCan4...,https://genius.com/Tiesto-and-tate-mcrae-10-35...
Preston Pablo,Flowers Need Rain,079mntueS2KWjxVqOHjG70,73.0,NaN,https://api.spotify.com/v1/tracks/079mntueS2KW...,https://genius.com/Preston-pablo-flowers-need-...
Rema,Calm Down (with Selena Gomez),0WtM2NBVQNNJLh6scP13H8,95.0,NaN,https://api.spotify.com/v1/tracks/0WtM2NBVQNNJ...,https://genius.com/Rema-calm-down-lyrics
TALK,Run Away to Mars,6G1Mz5yMgn0ydOlIvTrZ65,77.0,NaN,https://api.spotify.com/v1/tracks/6G1Mz5yMgn0y...,https://genius.com/Talk-can-run-away-to-mars-l...
Sam Smith,Gimme (feat. Koffee & Jessie Reyez),2GGN92tO0UbNAYrbMbQ4bf,74.0,NaN,https://api.spotify.com/v1/tracks/2GGN92tO0UbN...,https://genius.com/Sam-smith-koffee-and-jessie...


In [9]:
for i in tqdm(range(len(data))):
    url = data.at[i, 'lyricurl']

    try:
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'}) # This will disguise the bs4 object as a browser to avoid a 403 error

        webpage = urlopen(req).read()

        page_soup = soup(webpage, 'html.parser')

        content_list_items = page_soup.find_all('span', class_="ReferentFragmentdesktop__Highlight-sc-110r0d9-1 jAzSMw")
        lyric_string = ''

        for content_list in content_list_items:
            txt = content_list if type(content_list) == NavigableString else content_list.text
            lyric_string+=txt

        data.at[i, 'Lyrics'] = lyric_string
    
    except:
        
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'}) # This will disguise the bs4 object as a browser to avoid a 403 error

        webpage = urlopen(req).read()

        page_soup = soup(webpage, 'html.parser')

        content_list_items = page_soup.find_all('div', class_="Lyrics__Container-sc-1ynbvzw-6 YYrds")
        lyric_string = ''

        for content_list in content_list_items:
            txt = content_list if type(content_list) == NavigableString else content_list.text
            lyric_string+=txt

        data.at[i, 'Lyrics'] = lyric_string


  0%|                                                    | 0/76 [00:00<?, ?it/s]


ValueError: unknown url type: 'nan'

In [15]:
data.to_csv("data/song_data.csv", index=False)

In [20]:
df2 = data[data.Lyrics.isna() == True]
df2.head()

,Song,SongID,Popularity,Lyrics,href,lyricurl
Artist,,,,,,
Tiësto,10:35,6BePGk3eCan4FqaW2X8Qy3,90,NaN,https://api.spotify.com/v1/tracks/6BePGk3eCan4...,https://genius.com/Tiesto-and-tate-mcrae-10-35...
Preston Pablo,Flowers Need Rain,079mntueS2KWjxVqOHjG70,73,NaN,https://api.spotify.com/v1/tracks/079mntueS2KW...,https://genius.com/Preston-pablo-flowers-need-...
Rema,Calm Down (with Selena Gomez),0WtM2NBVQNNJLh6scP13H8,95,NaN,https://api.spotify.com/v1/tracks/0WtM2NBVQNNJ...,https://genius.com/Rema-calm-down-lyrics
TALK,Run Away to Mars,6G1Mz5yMgn0ydOlIvTrZ65,77,NaN,https://api.spotify.com/v1/tracks/6G1Mz5yMgn0y...,https://genius.com/Talk-can-run-away-to-mars-l...
Sam Smith,Gimme (feat. Koffee & Jessie Reyez),2GGN92tO0UbNAYrbMbQ4bf,74,NaN,https://api.spotify.com/v1/tracks/2GGN92tO0UbN...,https://genius.com/Sam-smith-koffee-and-jessie...


In [21]:
df2.shape

(34, 6)

In [22]:
df2.lyricurl[0]

'https://genius.com/Tiesto-and-tate-mcrae-10-35-mixed-lyrics'